In [3]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import cohen_kappa_score
from scipy.stats import mode
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from matplotlib import pyplot


In [4]:
from time import time
from tqdm import tqdm_notebook as tqdm

## Feature engineering taken from 
https://www.kaggle.com/shahules/xgboost-feature-selection-dsbowl

In [5]:
train = pd.read_csv('../data/raw/train.csv')

In [6]:
train_labels = pd.read_csv('../data/raw/train_labels.csv')

In [7]:
test = pd.read_csv('../data/raw/test.csv')

In [8]:
time_features=['month','hour','year','dayofweek','weekofyear']

In [9]:
def extract_time_features(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['month'] = df['timestamp'].dt.month
    df['hour'] = df['timestamp'].dt.hour
    df['year'] = df['timestamp'].dt.year
    df['dayofweek'] = df['timestamp'].dt.dayofweek
    df['weekofyear'] = df['timestamp'].dt.weekofyear
    return df

In [10]:
submission = pd.read_csv('../data/raw/sample_submission.csv')

In [11]:
train = extract_time_features(train)

In [12]:
def prepare_data(df):    
    df=df.drop('timestamp',axis=1)
    #df['timestamp']=pd.to_datetime(df['timestamp'])
    #df['hour_of_day']=df['timestamp'].map(lambda x : int(x.hour))
    

    join_one=pd.get_dummies(df[['event_code','installation_id','game_session']],
                            columns=['event_code']).groupby(['installation_id','game_session'],
                                                            as_index=False,sort=False).agg(sum)

    agg={'event_count':sum,'game_time':['sum','mean'],'event_id':'count'}

    join_two=df.drop(time_features,axis=1).groupby(['installation_id','game_session']
                                                   ,as_index=False,sort=False).agg(agg)
    
    join_two.columns= [' '.join(col).strip() for col in join_two.columns.values]
    

    join_three=df[['installation_id','game_session','type','world','title']].groupby(
                ['installation_id','game_session'],as_index=False,sort=False).first()
    
    join_four=df[time_features+['installation_id','game_session']].groupby(['installation_id',
                'game_session'],as_index=False,sort=False).agg(mode)[time_features].applymap(lambda x: x.mode[0])
    
    join_one=join_one.join(join_four)
    
    join_five=(join_one.join(join_two.drop(['installation_id','game_session'],axis=1))). \
                        join(join_three.drop(['installation_id','game_session'],axis=1))
    
    return join_five



In [ ]:

join_train=prepare_data(train)
cols=join_train.columns.to_list()[2:-3]
join_train[cols]=join_train[cols].astype('int16')

In [ ]:
cols=join_train.columns[2:-12].to_list()
cols.append('event_id count')
cols.append('installation_id')

In [ ]:
final_train=pd.merge(train_labels,join_train,on=['installation_id','game_session'],
                                         how='left').drop(['game_session'],axis=1)

In [ ]:
final_train['title'] = final_train['title_x']

In [ ]:
df=final_train[['event_count sum','game_time mean','game_time sum','installation_id']]. \
    groupby('installation_id',as_index=False,sort=False).agg('mean')

df_two=final_train[cols].groupby('installation_id',as_index=False,
                                 sort=False).agg('sum').drop('installation_id',axis=1)

df_three=final_train[['accuracy_group','title','type','world','installation_id']]. \
        groupby('installation_id',as_index=False,sort=False). \
        last().drop('installation_id',axis=1)

df_four=join_train[time_features+['installation_id']].groupby('installation_id',as_index=False,sort=False). \
        agg(mode)[time_features].applymap(lambda x : x.mode[0])



final_train=(df.join(df_two)).join(df_three.join(df_four)).drop('installation_id',axis=1)

In [ ]:
final_train

In [ ]:
final_train.to_csv('../data/processed/processed_train.csv')

## Try using improvement as a feature

In [19]:
final_train = pd.read_csv('../data/processed/processed_train.csv')

In [20]:
df = train
join_one=pd.get_dummies(df[['event_code','installation_id','game_session']],
                        columns=['event_code']).groupby(['installation_id','game_session'],
                                                        as_index=False,sort=False).agg(sum)

In [21]:
df = df.sort_values(['installation_id', 'timestamp'])

In [22]:
installation_ids = np.unique(df['installation_id'])

In [23]:
tid = installation_ids[0]

In [24]:
temp_df = df[df['installation_id'] == tid]

In [25]:
next_game_time = temp_df['game_time'].shift(-1)

In [26]:
temp_df['next_game_time'] = next_game_time

/projects/eerhan_prj/anaconda/envs/data/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [27]:
temp_df['improvement'] = temp_df['next_game_time'] - temp_df['game_time']

/projects/eerhan_prj/anaconda/envs/data/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [33]:
temp_df['improvement']

0           0.0
1           0.0
2          53.0
3        6919.0
4        3019.0
         ...   
1352        0.0
1353     1217.0
1354   -86619.0
1355        0.0
1356        NaN
Name: improvement, Length: 1357, dtype: float64

In [30]:
print('test')

test


In [53]:
new_dfs = []

In [ ]:
for installation_id in installation_ids:
    temp_df = df[df['installation_id'] == installation_id]
    next_game_time = temp_df['game_time'].shift(-1)
    temp_df['next_game_time'] = next_game_time
    temp_df['improvement'] = temp_df['next_game_time'] - temp_df['game_time']
    new_dfs.append(temp_df)

/projects/eerhan_prj/anaconda/envs/data/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/projects/eerhan_prj/anaconda/envs/data/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
new_df = pd.concat(new_dfs)